# Tutorial 9 - Changing the mesh

In [Tutorial 8](./tutorial-8-solver-options.ipynb) we saw how to change the solver options. In this tutorial we will change the mesh used in the simulation, and show how to investigate the influence of the mesh on the solution.

All models in PyBaMM have a default number of mesh points used in a simulation. However, depending on aspects like the operating conditions or the parameters, you may find you need to increase the number points in the mesh to obtain an accurate solution. On the other hand, you may find that you are able to decrease the number of mesh points and still obtain a solution with an acceptable degree of accuracy but with a lower computational time. 

It is always good practice to conduct a mesh refinement study, where you simulate the same problem with a finer mesh and compare the results. Here will show how to do this graphically, but in practice you may wish to do a more detailed calculation of the relative error.

In [ ]:
import pybamm

## Changing the number of points in the mesh

First we load a model, in this case the SPMe

In [ ]:
model = pybamm.lithium_ion.SPMe()

We can then look at the number of points that the models uses by default, which are stored as a dictionary whose keys are the variables for each domain:

In [ ]:
model.default_var_pts

Note how the number of points is a dictionary where the key is the name of the spatial variable, and the value the number of points in the discretisation of that variable. To run a simulation with a different number of points we can define our own dictionary 

In [ ]:
# create our dictionary
var_pts = {
    "x_n": 10,  # negative electrode
    "x_s": 10,  # separator
    "x_p": 10,  # positive electrode
    "r_n": 10,  # negative particle
    "r_p": 10,  # positive particle
}

and pass it as a keyword argument when creating a simulation

In [ ]:
sim = pybamm.Simulation(model, var_pts=var_pts)

We can then solve and plot the simulation as usual:

In [ ]:
sim.solve([0, 3600])
sim.plot()

## Conducting a mesh refinement study

In order to investigate the influence of the mesh on the solution we must solve the model multiple times, increasing the mesh resolution as we go. We first create a list of the number of points per domain we would like to use

In [ ]:
npts = [4, 8, 16, 32, 64]

and now we can loop over the list, creating and solving simulations as we go. The solutions are stored in the list `solutions`, similar to what we did in [Tutorial 2](./tutorial-2-compare-models.ipynb) for the various models.

In [ ]:
# choose model and parameters
model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues("Ecker2015")

# choose solver
solver = pybamm.CasadiSolver(mode="fast")

# loop over number of mesh points
solutions = []
for N in npts:
    var_pts = {
        "x_n": N,  # negative electrode
        "x_s": N,  # separator
        "x_p": N,  # positive electrode
        "r_n": N,  # negative particle
        "r_p": N,  # positive particle
    }
    sim = pybamm.Simulation(
        model, solver=solver, parameter_values=parameter_values, var_pts=var_pts
    )
    sim.solve([0, 3600])
    solutions.append(sim.solution)

We can now pass our list of solutions to the dynamic plot method, allowing use to see the influence of the mesh on the computed voltage. We pass our list of points using the `labels` keyword so that the plots are labeled with the number of points used in the simulation.

In [ ]:
pybamm.dynamic_plot(solutions, ["Voltage [V]"], labels=npts)

This notebook concludes the "Getting Started" series, that demonstrated all the main features of PyBaMM. You may now want to explore more advanced features, so take a look at all the [examples available](https://docs.pybamm.org/en/stable/source/examples/index.html) in our website.

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()